# Python Vechicle Simulator

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import matplotlib.pyplot as plt
import python_vehicle_simulator as pvs
from python_vehicle_simulator import plotVehicleStates, plotControls, simulate
import pandas as pd
from src.visualization.plot import track_plots, plot, captive_plot
import numpy as np
from src.prime_system import PrimeSystem
from python_vehicle_simulator.vehicles.tanker import tanker

In [ ]:
sampleTime = 1
N = 5000

In [ ]:
class Ship(tanker):
    
    n = 20
    index = np.linspace(0, N*sampleTime, n)
    deltas = pd.Series(np.deg2rad(np.linspace(-35, 35, n)), index=index)
        
    def replay_rudder(self, t):
        
        i = (self.deltas.index>=t).argmax()
        delta = self.deltas.iloc[i]
        return np.array([delta], float)
        
    

In [ ]:
ship = Ship()

In [ ]:
ship.n_c / 70.0  # propeller shaft speed (rps)
ship.controlMode='replay rudder'

nu0 = [7,0,0,0,0,0]
time, data = simulate(N, sampleTime, ship, nu0=nu0)

columns = [
'x0', 
'y0', 
'z0', 
'phi', 
'theta', 
'psi', 
'u', 
'v', 
'w', 
'p', 
'q', 
'r',
'delta_order',
'delta']

df = pd.DataFrame(index=time.flatten(), data=data, columns=columns)


In [ ]:
track_plots(dataframes={'sim':df}, lpp=ship.L, beam=ship.L/10);

In [ ]:
plot(dataframes={'sim':df});

In [ ]:
%reload_kedro

dataframes = {
#"raw_data" : catalog.load("tanker.simulation1.raw_data").loc[1000:2000],
"raw_data" : catalog.load("tanker.simulation1.data"),
    
#"ek" : catalog.load("tanker.initial.simulation1.data_ek_filter").loc[0:2000],
#"smooth" : catalog.load("tanker.initial.simulation1.data_ek_smooth"),
#"smooth2" : catalog.load("tanker.updated.simulation1.data_ek_smooth"),
"true_data" : catalog.load("tanker.simulation1.true_data"),
}

styles = {

    "true_data":{"style":'k-'},

}

ship_data = catalog.load("tanker.ship_data")


In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], N=10, plot_boats=True)

In [ ]:
plot(dataframes);

In [ ]:
%reload_kedro

ek = catalog.load("tanker.vmm_linear.ek")
ek.parameters['Ydelta']/=30
ek.parameters['Ndelta']/=30


In [ ]:
{key:value for key,value in ek.parameters.items() if value!=0}

In [ ]:
data = dataframes['true_data']
data['U'] = np.sqrt(data['u']**2 + data['v']**2)
df_sim = ek.simulate(data=data, input_columns=['delta','U'])

In [ ]:
time_window=[0,100]
plot({'true_data':data, 'sim':df_sim}, time_window=time_window);

In [ ]:
track_plots({'true_data':data, 'sim':df_sim}, lpp=ship_data['L'], beam=ship_data['B'], time_window=time_window, N=10)

In [ ]:
ship_data